In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX

warnings.filterwarnings('ignore')
os.environ['TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD'] = '1'

try:
    from neuralprophet import NeuralProphet
except ImportError as exc:
    raise ImportError(
        "NeuralProphet is not installed. Run: conda run -n ts-forecast pip install neuralprophet"
    ) from exc

# 3.4 Credit Reporting TSF: South vs Non-South (SARIMA, XGBoost, NeuralProphet)

In [ ]:
# Loading my 300k sample data
df = pd.read_parquet('../data/processed/cfpb_sample_300k.parquet')
print(f"Loaded {len(df):,} rows & {len(df.columns)} columns")

In [ ]:
# Focus only on Credit reporting product and create South vs Non-South segments
df_cr = df[df['Product'].str.contains('Credit reporting', case=False, na=False)].copy()
df_cr['segment'] = np.where(df_cr['region'].eq('South'), 'South', 'Non-South')

quarterly_counts = (
    df_cr.groupby(['year_quarter', 'segment'])
    .size()
    .unstack(fill_value=0)
    .sort_index()
)

quarterly_counts.index = pd.PeriodIndex(quarterly_counts.index, freq='Q')

print('Credit reporting rows:', f"{len(df_cr):,}")
print('Available segments:', quarterly_counts.columns.tolist())
quarterly_counts.tail()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
for seg in ['South', 'Non-South']:
    if seg in quarterly_counts.columns:
        s = quarterly_counts[seg].copy()
        s.index = s.index.to_timestamp(how='end')
        ax.plot(s.index, s.values, label=seg)

ax.set_title('Credit Reporting Complaints by Quarter: South vs Non-South')
ax.set_ylabel('Complaint count')
ax.legend()
plt.tight_layout()
plt.show()

### 3.4.1 Modeling Helpers

In [ ]:
def split_series(series: pd.Series):
    n = len(series)
    test_size = 8 if n >= 24 else max(4, int(round(n * 0.2)))
    test_size = min(test_size, n - 8) if n > 12 else max(1, n // 3)
    train = series.iloc[:-test_size].copy()
    test = series.iloc[-test_size:].copy()
    return train, test

def run_sarima(series: pd.Series, steps: int = 4):
    train, test = split_series(series)

    try:
        model = SARIMAX(train, order=(1, 1, 0), seasonal_order=(1, 1, 0, 4), trend='c')
        fitted = model.fit(disp=False)
    except Exception:
        model = SARIMAX(train, order=(1, 1, 0), seasonal_order=(0, 0, 0, 0), trend='c')
        fitted = model.fit(disp=False)

    test_pred = fitted.forecast(steps=len(test))
    test_pred.index = test.index

    holdout_mae = mean_absolute_error(test, test_pred)
    holdout_rmse = np.sqrt(mean_squared_error(test, test_pred))

    final_model = SARIMAX(series, order=(1, 1, 0), seasonal_order=(1, 1, 0, 4), trend='c')
    final_fit = final_model.fit(disp=False)
    fc = final_fit.forecast(steps=steps)

    fc_index = pd.period_range(start=series.index[-1] + 1, periods=steps, freq='Q')
    fc = pd.Series(fc.values, index=fc_index, name='forecast')

    return {
        'test_actual': test,
        'test_pred': test_pred,
        'forecast': fc,
        'mae': holdout_mae,
        'rmse': holdout_rmse
    }

def make_xgb_features(series: pd.Series):
    df_feat = pd.DataFrame({'y': series.values}, index=series.index.to_timestamp(how='end'))
    for lag in [1, 2, 3, 4]:
        df_feat[f'lag_{lag}'] = df_feat['y'].shift(lag)
    df_feat['rolling_mean_2'] = df_feat['y'].shift(1).rolling(2).mean()
    df_feat['rolling_mean_4'] = df_feat['y'].shift(1).rolling(4).mean()
    df_feat['quarter'] = df_feat.index.quarter
    df_feat['t'] = np.arange(len(df_feat))
    df_feat = df_feat.dropna().copy()

    feature_cols = ['lag_1', 'lag_2', 'lag_3', 'lag_4', 'rolling_mean_2', 'rolling_mean_4', 'quarter', 't']
    X = df_feat[feature_cols]
    y = df_feat['y']
    return df_feat, X, y, feature_cols

def run_xgb(series: pd.Series, steps: int = 4):
    train, test = split_series(series)

    train_feat, X_train, y_train, feature_cols = make_xgb_features(train)
    full_feat, X_full, y_full, _ = make_xgb_features(series)

    test_feat, X_test, y_test, _ = make_xgb_features(pd.concat([train, test]))
    test_start_ts = test.index[0].to_timestamp(how='end')
    test_mask = test_feat.index >= test_start_ts
    X_test_eval = X_test.loc[test_mask]
    y_test_eval = y_test.loc[test_mask]

    model = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=500,
        learning_rate=0.03,
        max_depth=3,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=42
    )
    model.fit(X_train, y_train)

    pred_vals = model.predict(X_test_eval)
    test_pred = pd.Series(pred_vals, index=test.index, name='xgb_pred')

    holdout_mae = mean_absolute_error(test, test_pred)
    holdout_rmse = np.sqrt(mean_squared_error(test, test_pred))

    final_model = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=500,
        learning_rate=0.03,
        max_depth=3,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=42
    )
    final_model.fit(X_full, y_full)

    history = full_feat['y'].tolist()
    future_index = pd.date_range(start=full_feat.index[-1] + pd.offsets.QuarterEnd(1), periods=steps, freq='QE')
    future_preds = []

    for dt in future_index:
        row = pd.DataFrame({
            'lag_1': [history[-1]],
            'lag_2': [history[-2]],
            'lag_3': [history[-3]],
            'lag_4': [history[-4]],
            'rolling_mean_2': [np.mean(history[-2:])],
            'rolling_mean_4': [np.mean(history[-4:])],
            'quarter': [dt.quarter],
            't': [len(history)]
        }, index=[dt])

        yhat = float(final_model.predict(row[feature_cols])[0])
        future_preds.append(yhat)
        history.append(yhat)

    fc_index = pd.PeriodIndex(future_index, freq='Q')
    forecast = pd.Series(future_preds, index=fc_index, name='forecast')

    return {
        'test_actual': test,
        'test_pred': test_pred,
        'forecast': forecast,
        'mae': holdout_mae,
        'rmse': holdout_rmse
    }

def run_neuralprophet(series: pd.Series, steps: int = 4):
    train, test = split_series(series)

    np_train = pd.DataFrame({'ds': train.index.to_timestamp(how='start'), 'y': train.values})
    np_test = pd.DataFrame({'ds': test.index.to_timestamp(how='start'), 'y': test.values})
    np_full = pd.DataFrame({'ds': series.index.to_timestamp(how='start'), 'y': series.values})

    model = NeuralProphet(
        n_lags=4,
        n_forecasts=1,
        changepoints_range=0.9,
        yearly_seasonality=False,
        weekly_seasonality=False,
        daily_seasonality=False,
        seasonality_mode='additive'
    )

    _ = model.fit(np_train, freq='QS', progress='off')
    future_test = model.make_future_dataframe(np_train, periods=len(np_test), n_historic_predictions=True)
    forecast_test = model.predict(future_test)

    pred_test = (
        forecast_test[['ds', 'yhat1']]
        .dropna()
        .merge(np_test[['ds', 'y']], on='ds', how='inner')
        .rename(columns={'yhat1': 'y_pred', 'y': 'y_true'})
    )

    pred_test = pred_test.tail(len(test))
    test_pred = pd.Series(pred_test['y_pred'].values, index=test.index, name='np_pred')

    holdout_mae = mean_absolute_error(test, test_pred)
    holdout_rmse = np.sqrt(mean_squared_error(test, test_pred))

    final_model = NeuralProphet(
        n_lags=4,
        n_forecasts=1,
        changepoints_range=0.9,
        yearly_seasonality=False,
        weekly_seasonality=False,
        daily_seasonality=False,
        seasonality_mode='additive'
    )

    _ = final_model.fit(np_full, freq='QS', progress='off')
    future_full = final_model.make_future_dataframe(np_full, periods=steps, n_historic_predictions=True)
    forecast_full = final_model.predict(future_full)

    fc_vals = forecast_full[['ds', 'yhat1']].tail(steps).copy()
    fc_index = pd.PeriodIndex(fc_vals['ds'], freq='Q')
    forecast = pd.Series(fc_vals['yhat1'].values, index=fc_index, name='forecast')

    return {
        'test_actual': test,
        'test_pred': test_pred,
        'forecast': forecast,
        'mae': holdout_mae,
        'rmse': holdout_rmse
    }

### 3.4.2 Run All 3 Models by Segment

In [ ]:
segments = ['South', 'Non-South']
results = []
model_outputs = {}

for segment in segments:
    if segment not in quarterly_counts.columns:
        continue

    series = quarterly_counts[segment].astype(float)

    sarima_out = run_sarima(series, steps=4)
    xgb_out = run_xgb(series, steps=4)
    np_out = run_neuralprophet(series, steps=4)

    model_outputs[(segment, 'SARIMA')] = sarima_out
    model_outputs[(segment, 'XGBoost')] = xgb_out
    model_outputs[(segment, 'NeuralProphet')] = np_out

    for model_name, out in [('SARIMA', sarima_out), ('XGBoost', xgb_out), ('NeuralProphet', np_out)]:
        results.append({
            'segment': segment,
            'model': model_name,
            'holdout_mae': round(out['mae'], 1),
            'holdout_rmse': round(out['rmse'], 1),
            'mae_pct_of_mean': round((out['mae'] / series.mean()) * 100, 1)
        })

results_df = pd.DataFrame(results).sort_values(['segment', 'holdout_mae'])
results_df

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

for i, segment in enumerate(['South', 'Non-South']):
    ax = axes[i]
    s = quarterly_counts[segment].astype(float)
    s_test = next(v['test_actual'] for (seg, mdl), v in model_outputs.items() if seg == segment and mdl == 'SARIMA')

    ax.plot(s.index.to_timestamp(how='end'), s.values, label='Actual', color='black', alpha=0.6)

    for model_name, color in [('SARIMA', 'steelblue'), ('XGBoost', 'tomato'), ('NeuralProphet', 'seagreen')]:
        pred = model_outputs[(segment, model_name)]['test_pred']
        ax.plot(pred.index.to_timestamp(how='end'), pred.values, label=f'{model_name} holdout', color=color)

    ax.set_title(f'Holdout Comparison - {segment}')
    ax.legend(loc='upper left', ncol=2)

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=False)

for i, segment in enumerate(['South', 'Non-South']):
    ax = axes[i]
    obs = quarterly_counts[segment].astype(float)
    ax.plot(obs.index.to_timestamp(how='end'), obs.values, label='Observed', color='black', alpha=0.6)

    for model_name, color in [('SARIMA', 'steelblue'), ('XGBoost', 'tomato'), ('NeuralProphet', 'seagreen')]:
        fc = model_outputs[(segment, model_name)]['forecast']
        ax.plot(fc.index.to_timestamp(how='end'), fc.values, marker='o', label=f'{model_name} forecast', color=color)

    ax.set_title(f'Next 4 Quarters Forecast - {segment}')
    ax.legend(loc='upper left', ncol=2)

plt.tight_layout()
os.makedirs('../reports/figures', exist_ok=True)
plt.savefig('../reports/figures/credit_reporting_south_vs_others_tsf.png', dpi=150)
plt.show()

### 3.4.3 Summary Table (Best Model Per Segment)

In [ ]:
best_by_segment = results_df.sort_values(['segment', 'holdout_mae']).groupby('segment', as_index=False).first()
best_by_segment

This notebook compares SARIMA, XGBoost, and NeuralProphet for Credit reporting complaints in South vs Non-South segments using the same holdout setup and forecast horizon.

- `results_df` contains full model-by-segment metrics.
- `best_by_segment` shows the lowest-MAE model for each segment.
- Forecast figure is saved to `../reports/figures/credit_reporting_south_vs_others_tsf.png`.